# Fluent Python
- by Luciano Ramalho
- https://www.amazon.com/Fluent-Python-Concise-Effective-Programming/dp/1491946008

## Chapter 1: python data model
1. Dunder method (magic method)
    - Python built-in method, standard libs are built upon dunder method.
    - Define dunder method for your class. Get all functionalities for free.
    
    


In [1]:
import random


class School:
    def __init__(self, students):
        self.students = list(students)

    def __len__(self):
        return len(self.students)

    def __getitem__(self, i):
        return self.students[i]


school = School(["Alice", "Bob", "Charlie"])
print(f"{len(school)=}")
print(f"{school[0:2]=}")
print(f"{[student for student in school]=}")
print(f"{random.choice(school)=}")

len(school)=3
school[0:2]=['Alice', 'Bob']
[student for student in school]=['Alice', 'Bob', 'Charlie']
random.choice(school)='Bob'


## Chapter 2: an array of sequences
1. list operations: `+`, `*`
2. array stores packed bytes, which is more effective than list
3. Use memoryview() to inspect memory buffer with cast()

In [3]:
import array

print(f"{[1,2,3] + [4,5,6]=}")
print(f"{[1,2,3] * 3=}")
arr = array.array("i", [1, 2, 3])
print(f"{arr=}")

[1,2,3] + [4,5,6]=[1, 2, 3, 4, 5, 6]
[1,2,3] * 3=[1, 2, 3, 1, 2, 3, 1, 2, 3]
arr=array('i', [1, 2, 3])


## Chapter 3: dictionaries and sets
1. hashable
    - Hashable-ness can be used as dict/set key
    - Hashable requires to define __hash__() and __eq__(). Equal objects much have the same hash.
    - Custom classes are by default hashable. __hash__() returns id(), and __eq__() returns False.
2. Use defaultdict and setdefault to handle missing values
3. OrderedDict, Counter are handy dict variants
4. Handy set opertions for & and |

In [7]:
from collections import defaultdict


class Foo:
    def __init__(self, x):
        self.x = x


foo = Foo(42)
d = {foo: "bar"}
print(f"{d[foo]=}")


class Bar:
    def __init__(self, x):
        self.x = x

    def __hash__(self):
        return hash(self.x)

    def __eq__(self, other):
        return self.x == other.x


bar = Bar(42)
d = {bar: "baz"}
print(f"{d[bar]=}")

res = defaultdict(lambda: defaultdict(list))
res["foo"]["bar"].append(42)
res["foo"]["bar"].append(23)
res["foo"]["baz"].append(7)
print(f"{res=}")

s1 = {1, 2, 3}
s2 = {3, 4, 5}
print(f"{s1 | s2=}")
print(f"{s1 & s2=}")

d[foo]='bar'
d[bar]='baz'
res=defaultdict(<function <lambda> at 0x10da53380>, {'foo': defaultdict(<class 'list'>, {'bar': [42, 23], 'baz': [7]})})
s1 | s2={1, 2, 3, 4, 5}
s1 & s2={3}


## Chapter 4. str vs bytes
1. Unicode: separation of code point and byte representation
    - Encode: code point to bytes
    - Decode: bytes to code point
    - Str in RAM representation is Python implementation details. Usually some memory efficient format
2. Bytes display
    - Printable ASCII bytes: displayed as is
    - Special chars, e.g. tab, newline: using escape sequences, e.g. \t, \n, etc
    - Other bytes: hexadecimal escape sequence, e.g. \x00
3. Unicode sandwidth
    - Decode bytes on input, process text only, encode text on output
    - open() handles encoding/decoding automatically 
        - w/r: open files in text mode, with default uft-8 encoding/decoding. str in and str out. 
        - raises exception if, say, write bytes
        - wb/rb: open files in byte mode. expect bytes input/output

In [6]:
import encodings

s = "你好"
b = s.encode("utf-8")
s_ = b.decode("utf-8")
print(f"{s_=}")
print(f"{b=}")

b = b"abc\t\n\x01"
print(f"{b=}")

with open("test.txt", "wb") as f:
    f.write("Hello, World!\n".encode("utf-8"))
with open("test.txt", "rb") as f:
    print(f"{f.read()=}")

s_='你好'
b=b'\xe4\xbd\xa0\xe5\xa5\xbd'
b=b'abc\t\n\x01'
f.read()=b'Hello, World!\n'


## Chapter 5. First Class Functions
0. First class object:
    - created at runtime, passed as an argument to a function, returned as a result from a function
1. dir() vs __dict__(): 
    - dir() returns all attributes?
    - __dict__() returns user attributes assigned to it?
2. Inspection
    - __names__, __doc__, __annotation__, __code__, etc
    - inspect module
3. Handy higher-order functions
    - partial: freeze arguments

In [8]:
import functools


def fn(a, b, c):
    print(f"{a=} {b=} {c=}")
    return a + b + c


fn_by_7 = functools.partial(fn, c=7)
print(f"{fn_by_7(1, 2)=}")

a=1 b=2 c=7
fn_by_7(1, 2)=10


## Chapter 7. Decorators and Closures
1. Variable scope
    - Python compiles the body of the function before execution
    - If a variable is assigned to a value, Python treats it as a new variable
    - If a variable is accessed only, it look up outer scopes as a reference
    - Use dis() to disassemble a function and inspect bytecode
2. Closure: a function with an extended scope 
    - Access non-global variables that defined out of its body
    - Called free variables
3. nonlocal: declare reference to a free variable
4. Decorator & Parameterized Decorator
    - Use functools.wraps to copy relavant attributes
    - Parameterized decorator needs to be a decorator factory
    - Another handy functools lib, lru_cache

In [11]:
import functools


def moving_avg():
    sum = 0.0
    n = 0

    def inner(x):
        nonlocal sum, n
        sum += x
        n += 1
        return sum / n

    return inner


ma = moving_avg()
print(f"{ma(1)=}")
print(f"{ma(2)=}")


def lru_cache(maxsize=128):
    def decorator(fn):
        cache = {}

        @functools.wraps(fn)
        def inner(*args, **kwargs):
            key = (args, tuple(kwargs.items()))
            if key in cache:
                return cache[key]
            result = fn(*args, **kwargs)
            if len(cache) >= maxsize:
                cache.pop(next(iter(cache)))
            cache[key] = result
            return result

        return inner

    return decorator


@lru_cache(maxsize=2)
def fn(x):
    print(f"Calculating {x}")
    return x * x


fn(1)
fn(2)
fn(1)
fn(3)
fn(1)

ma(1)=1.0
ma(2)=1.5
Calculating 1
Calculating 2
Calculating 3
Calculating 1


1

## Chapter 8. Object Memory Management
1. Identity, equality, and alias
    - `is` vs `==`
2. Copy vs deepcopy
    - Shallow copy by default
    - Shallow copy creates alias for each attribute of the object
        - For list, it creates alias for each element
        - For dict, it creates alias for each kv
    - deepcopy recursively copies everything
3. Function parameters as references
    - Mutable types as parameter defaults is error prone, e.g. def foo(l=[])
        - All class/function instances share the same default param value
    - Use copy instead of assign to store argument as member variable, e.g.
        def foo(self, l):
            self._l = list(l)
4. Gargabe Collection
    - del deletes names, not objects
    - Objects are freed either, 1) refcount reaches zero, immediately destroyed, 2) reference cycle detection when gc.collect()
    - Python console automatically bind _ variable to the result of expression that are not None

In [15]:
import copy

l = [1, 2, [3, 4]]
l2 = copy.copy(l)
l3 = copy.deepcopy(l)
l.append(5)
l[2].append(6)
print(f"{l=}")
print(f"{l2=}")
print(f"{l3=}")

l=[1, 2, [3, 4, 6], 5]
l2=[1, 2, [3, 4, 6]]
l3=[1, 2, [3, 4]]


## Chapter 9. A Pythonic Object (more dunder methods)
1. Classmethod vs Staticmethod: 
    - Classmethod: commonly used as alternative constructors
    - Staticmethod: no good reason of existance
2. More dunder methods: __format__(), __hash__()
    - __slots__: 1) efficient memory format, 2) forbid extra attributes definitions

In [18]:
class Foo:
    __slots__ = ["x", "y"]

    def __init__(self, x, y):
        self.x = x
        self.y = y


class Bar:
    def __init__(self, x, y):
        self.x = x
        self.y = y


foo = Foo(1, 2)
try:
    foo.z = 3
except AttributeError as e:
    print(e)
bar = Bar(1, 2)
bar.z = 3

'Foo' object has no attribute 'z'


## Chapter 10 & 11. More and more dunner methods
1. Duck-typing: don't check type. Check behaviors.
2. Monkey patching: changing a class or module at runtime, without touching the source code.

## Chapter 14. Iterables, Iterators, and Generators
1. For-in-loop under the hook  
    - calls iter(x) over an object to get the iterator
    - iter() checks if __iter__() is implemented, or fallback to __getitem__(), or TypeError
    - repeatedly calls next(it), until StopIteration exception
2. Iterable vs Iterator
    - Iterable interface implements __iter__() method which returns a iterator, (or implements __getitem__())
    - Iterator inteface implements __next__() to return the next available item (or raise StopIteration), 
        and __iter__() to return itself, which allows iterators to be used where an iterable is expected
    Iterators are iterable. Iterables may not be iterators.
3. Generator vs Iterator
    - Functional wise, every generator is a iterator, which implements iterator interface (__next__ and __iter__).
    - Conceptual wise, iterator retrieves items from an existing inventory, whereas generator creates new things.
    - In many cases, people don't strictly distinsh iterator and generator.
4. itertools.count(0, 1)

In [1]:
class StudentsIterator:
    def __init__(self, students):
        self.students = list(students)
        self.i = 0

    def __next__(self):
        if self.i < len(self.students):
            student = self.students[self.i]
            self.i += 1
            return student
        raise StopIteration

    def __iter__(self):
        return self


class Students:
    def __init__(self, students):
        self.students = students

    def __iter__(self):
        return StudentsIterator(self.students)


students = Students(["Alice", "Bob", "Charlie"])
for student in students:
    print(student)

Alice
Bob
Charlie


## Chapter 15. Contgext Manasger
1. Implement `__enter__()` and `__exit__()` for context manager interface.
2. Use `@contextlib.contextmanager` decorator with yield.
3. Remember to include try final, otherwise exception raised in the body of with block, without restoring the state.

In [3]:
from contextlib import contextmanager


@contextmanager
def latency(tag):
    import time

    start = time.time()
    try:
        yield
    finally:
        print(f"{tag} took {time.time() - start:.2f}s")


with latency("sleep 1s"):
    import time

    time.sleep(1)

sleep 1s took 1.01s


## Chapter 16. Coroutines
1. Coroutines for Cooperative Multitask
2. Coroutine has 4 state: `created`, `running`, `suspended`, `closed`
3. Push and pull values from coroutine: `x = yield y`
4. Major interface: `next()`, `send()`, `close()`, `throw()`
5. `yield from`: a syntax to allow the client to directly drive subgenerator directly, effectively bypass delegating generators

In [6]:
import random

options = ("rock", "paper", "scissors")


def player(name):
    opponent = yield
    while True:
        if opponent is None:
            break
        option = options[(options.index(opponent) + 1) % 3]
        print(f"{name} plays {option}")
        opponent = yield option


p1 = player("Alice")
next(p1)
p2 = player("Bob")
next(p2)
option = p1.send(random.choice(options))
for round in range(3):
    option = p1.send(option)
    option = p2.send(option)


Alice plays paper
Alice plays scissors
Bob plays rock
Alice plays paper
Bob plays scissors
Alice plays rock
Bob plays paper
